In [20]:
import os
import cv2
import numpy as np

In [21]:
# === Konfigurasi ===
input_folder = "Satriadata/Train/"
output_folder = "Train_preprocessed/"
target_size = 640  # YOLOv8 input biasanya 640x640

os.makedirs(output_folder, exist_ok=True)


In [22]:
# Fungsi: letterbox resize (biar tidak merusak aspek rasio)
def letterbox_resize(image, target_size=640):
    h, w = image.shape[:2]
    scale = target_size / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)
    resized = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
    
    # Buat canvas hitam lalu tempelkan di tengah
    canvas = np.full((target_size, target_size, 3), 128, dtype=np.uint8)
    top = (target_size - new_h) // 2
    left = (target_size - new_w) // 2
    canvas[top:top+new_h, left:left+new_w] = resized
    return canvas

# Fungsi: CLAHE untuk perbaikan kontras
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# Fungsi: Gamma correction
def adjust_gamma(image, gamma=1.2):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# Fungsi utama preprocessing
def preprocess_image(img):
    img = letterbox_resize(img, target_size)
    img = apply_clahe(img)
    img = adjust_gamma(img, gamma=1.2)
    img = cv2.bilateralFilter(img, 9, 75, 75)  # denoise tanpa hilangkan tepi
    return img

# Proses semua gambar di folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith((".jpg", ".jpeg", ".png")):
        path = os.path.join(input_folder, filename)
        img = cv2.imread(path)
        if img is None:
            print(f"[SKIP] Gagal membaca {filename}")
            continue

        processed = preprocess_image(img)
        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, processed)
        print(f"[OK] {filename} diproses & disimpan ke {save_path}")

print("\n✅ Semua gambar selesai dipreprocessing!")

[OK] DataTrain1.png diproses & disimpan ke Train_preprocessed/DataTrain1.png
[OK] DataTrain10.png diproses & disimpan ke Train_preprocessed/DataTrain10.png
[OK] DataTrain100.png diproses & disimpan ke Train_preprocessed/DataTrain100.png
[OK] DataTrain101.png diproses & disimpan ke Train_preprocessed/DataTrain101.png
[OK] DataTrain102.png diproses & disimpan ke Train_preprocessed/DataTrain102.png
[OK] DataTrain103.png diproses & disimpan ke Train_preprocessed/DataTrain103.png
[OK] DataTrain104.png diproses & disimpan ke Train_preprocessed/DataTrain104.png
[OK] DataTrain105.png diproses & disimpan ke Train_preprocessed/DataTrain105.png
[OK] DataTrain106.png diproses & disimpan ke Train_preprocessed/DataTrain106.png
[OK] DataTrain107.png diproses & disimpan ke Train_preprocessed/DataTrain107.png
[OK] DataTrain108.png diproses & disimpan ke Train_preprocessed/DataTrain108.png
[OK] DataTrain109.png diproses & disimpan ke Train_preprocessed/DataTrain109.png
[OK] DataTrain11.png diproses & di